In [9]:
import pandas as pd

df = pd.read_csv('filtered_weather_yearly_2012.csv')

# 총데이터 개수 8633
df_null = (df == -9.0).sum()
print (df_null)

YYMMDDHHMI       0
WD               0
WS               0
TA              13
TD              14
HM               0
RN            7762
SD_TOT        7965
CA_TOT        1894
CA_MID        1894
VS            1894
TS              13
SI            3910
PS               0
PA               0
dtype: int64


In [ ]:
# 기상청 날씨데이터 전처리 - kma_weather_ready.csv 파일 생성

import pandas as pd
import glob
import numpy as np

file_path = 'filtered_weather_yearly_*.csv'

csv_files = glob.glob(file_path)
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# print(df)

df['YYMMDDHHMI'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
df = df.sort_values(by='YYMMDDHHMI', ascending=True)

# 날짜 관련 열 추가 함수
def timeshape(df):
    df['YYMMDDHHMI'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    
    # 년, 월, 일, 시, 요일 열 추가
    df['year'] = df['YYMMDDHHMI'].dt.year
    df['month'] = df['YYMMDDHHMI'].dt.month
    df['day'] = df['YYMMDDHHMI'].dt.day
    df['hour'] = df['YYMMDDHHMI'].dt.hour
    df['weekday'] = df['YYMMDDHHMI'].dt.weekday  # 월요일=0, 일요일=6

    return df

# 날짜 관련 열 추가 후 반환
df = timeshape(df)

# 날짜 범위 생성 (2012년 1월 1일부터 2024년 12월 31일까지, 시간 간격 1시간)
date_range = pd.date_range(start='2012-01-01 00:00:00', end='2024-12-31 23:00:00', freq='h')
all_dates_df = pd.DataFrame(date_range, columns=['YYMMDDHHMI'])

# 이미 존재하는 df의 날짜 형식 'YYYY-MM-DD HH:MM:SS'로 맞춰주기 위해 변환
df['YYMMDDHHMI'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y-%m-%d %H:%M:%S')

# 누락된 날짜 찾기
no_date_df = all_dates_df[~all_dates_df['YYMMDDHHMI'].isin(df['YYMMDDHHMI'])]

# 누락된 날짜 데이터를 동일한 형식으로 변환
no_date_df.loc[:, 'YYMMDDHHMI'] = pd.to_datetime(no_date_df['YYMMDDHHMI'], format='%Y-%m-%d %H:%M:%S')


# print(no_date_df)







def add_missing_data(df, no_date_df):
    # 누락된 데이터에 -9.0 값 채우기 (특정 컬럼만)
    fill_col = ['WD', 'WS', 'HM', 'CA_MID', 'VS', 'PS', 'PA', 'RN', 'SD_TOT', 'CA_TOT', 'SI']
    temp_col = ['TA', 'TD', 'TS']
    
    # 월별 시간대별 평균을 계산 (temp_col에 해당하는 열들에 대해)
    temp_col_mean = {}
    for col in temp_col:
        temp_col_mean[col] = df.groupby(['month', 'hour'])[col].mean()  # -9.0을 제외하고 평균 계산
    
    for index, row in no_date_df.iterrows():
        new_row = {'YYMMDDHHMI': row['YYMMDDHHMI']}  # 새로운 날짜값
        
        # 'YYMMDDHHMI'를 datetime 형식으로 변환
        new_row['YYMMDDHHMI'] = pd.to_datetime(new_row['YYMMDDHHMI'], format='%Y%m%d%H%M')
        
        # timeshape 함수로 날짜 관련 열 추가
        new_row_df = pd.DataFrame([new_row])
        new_row_df = timeshape(new_row_df).iloc[0]  # timeshape 함수로 year, month, day, hour, weekday 추가
        
        # new_row에 timeshape에서 생성된 값들을 합침
        for col in ['year', 'month', 'day', 'hour', 'weekday']:
            new_row[col] = new_row_df[col]
        
        # 특정 컬럼들만 -9.0으로 채움
        for col in fill_col:
            new_row[col] = -9.0
        
        # temp_col에 대해서는 월별 시간대별 평균을 대입
        for col in temp_col:
            # 해당 월과 시간대의 평균값을 가져와서 대입
            mean_value = temp_col_mean[col].get((new_row['month'], new_row['hour']), np.nan)
            new_row[col] = mean_value
        
        # 나머지 컬럼은 건드리지 않음, 원본 df에서 그대로 복사
        for col in df.columns:
            if col not in fill_col and col != 'YYMMDDHHMI' and col not in temp_col:
                new_row[col] = np.nan  # 다른 컬럼은 결측치 처리 (혹은 적절한 초기값으로 채울 수도 있음)

        # df에 추가
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    # 'YYMMDDHHMI' 기준으로 다시 정렬
    df = df.sort_values(by='YYMMDDHHMI').reset_index(drop=True)
    
    # 최종적으로 'YYMMDDHHMI' 열을 'year', 'month', 'day', 'hour', 'weekday'로 변환
    df = timeshape(df)  # df에 날짜 관련 열 추가

    return df

df = add_missing_data(df, no_date_df)







column_order = [
    'year', 'month', 'day', 'hour', 'weekday',
    'WD','WS','TA','TD','HM','RN','SD_TOT',
    'CA_TOT','CA_MID','VS','TS','SI','PS','PA'
]

df = df[column_order]

# print(df)







# 출력 확인
# print(df.head())

# df.to_csv('weather_data_all.csv', index=False) # df 파일 필요시 저장

weather_mean = ['WD','WS','HM','CA_MID','VS','PS','PA']
mean_values = df[weather_mean].apply(lambda x: x[x != -9.0].mean()) 
for col in weather_mean:
    df[col] = df[col].replace(-9.0, mean_values[col])

weather_zero = ['RN','SD_TOT']
for col in weather_zero:  
    df[col] = df[col].replace(-9.0, 0)

# CA_TOT 결측치 처리 (해당 월 평균값)
def fill_ca_tot(row):
    if row['CA_TOT'] == -9.0:
        mean_ca_tot = df[df['month'] == row['month']]['CA_TOT'].replace(-9.0, np.nan).mean()
        return mean_ca_tot
    else:
        return row['CA_TOT']

df['CA_TOT'] = df.apply(fill_ca_tot, axis=1)

def fill_si(row):
    # 하절기: 4월 ~ 9월, 동절기: 10월 ~ 3월
    summer = 4 <= row['month'] <= 9
    winter = not summer

    # 결측치 SI 처리
    if row['SI'] == -9.0:
        # 하절기: 6시~21시 결측치는 평균값으로 대체
        if summer and 6 <= row['hour'] < 21:
            mean_si = df[(df['hour'] >= 6) & (df['hour'] < 21) & (df['month'] == row['month'])]['SI'].replace(-9.0, np.nan).mean()
            return mean_si
        
        # 동절기: 8시~19시 결측치는 평균값으로 대체
        if winter and 8 <= row['hour'] < 19:
            mean_si = df[(df['hour'] >= 8) & (df['hour'] < 19) & (df['month'] == row['month'])]['SI'].replace(-9.0, np.nan).mean()
            return mean_si
        
        # 나머지 밤 시간대 결측치는 0으로 대체
        else:
            return 0
    
    # 결측치가 아니면 원래 값 유지
    return row['SI']

df['SI'] = df.apply(fill_si, axis=1)

df.columns = df.columns.str.lower()

# df_null = (df == -9.0).sum()
# print (df_null)
    
df.to_csv('kma_weather_ready.csv', index=False)

print(df.head())

# filtered_data = df[(df['year'] == 2023) & 
#                        (df['month'] == 12) & 
#                        (df['day'] == 10) & 
#                        (df['hour'] == 6)]

# # 필터링된 데이터 출력
# print(filtered_data)

   year  month  day  hour  weekday         wd        ws        ta         td  \
0  2012      1    1     0        6  19.020319  2.363844 -2.732564  -9.881282   
1  2012      1    1     1        6  19.020319  2.363844 -3.036154  -9.984615   
2  2012      1    1     2        6  19.020319  2.363844 -3.336923 -10.133333   
3  2012      1    1     3        6  19.020319  2.363844 -3.596667 -10.227179   
4  2012      1    1     4        6  19.020319  2.363844 -3.858974 -10.341026   

          hm   rn  sd_tot    ca_tot    ca_mid           vs        ts   si  \
0  61.210913  0.0     0.0  3.865797  3.157111  1576.233283 -2.769231  0.0   
1  61.210913  0.0     0.0  3.865797  3.157111  1576.233283 -2.986410  0.0   
2  61.210913  0.0     0.0  3.865797  3.157111  1576.233283 -3.214615  0.0   
3  61.210913  0.0     0.0  3.865797  3.157111  1576.233283 -3.430769  0.0   
4  61.210913  0.0     0.0  3.865797  3.157111  1576.233283 -3.618462  0.0   

            ps           pa  
0  1016.222858  1005.96829